In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, log_loss
import operator
import json
from IPython import display
import os
import warnings
import itertools
np.random.seed(0)
warnings.filterwarnings("ignore")
THRESHOLD = 4

In [2]:
# read data from file
# train = pd.read_csv("../input/train.csv") 
# test = pd.read_csv("../input/test.csv")
train = pd.read_csv("train.csv") 
test = pd.read_csv("test.csv")


# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 355
Number of features in train: 99
Number of data points in test: 119
Number of features in test: 98


In [3]:
def data_clean(data):
    
    # Let's first remove all missing value features
    columns_to_remove = ['Also Known As','Applications','Audio Features','Bezel-less display'
                         'Browser','Build Material','Co-Processor','Browser'
                         'Display Colour','Mobile High-Definition Link(MHL)',
                         'Music', 'Email','Fingerprint Sensor Position',
                         'Games','HDMI','Heart Rate Monitor','IRIS Scanner', 
                         'Optical Image Stabilisation','Other Facilities',
                         'Phone Book','Physical Aperture','Quick Charging',
                         'Ring Tone','Ruggedness','SAR Value','SIM 3','SMS',
                         'Screen Protection','Screen to Body Ratio (claimed by the brand)',
                         'Sensor','Software Based Aperture', 'Special Features',
                         'Standby time','Stylus','TalkTime', 'USB Type-C',
                         'Video Player', 'Video Recording Features','Waterproof',
                         'Wireless Charging','USB OTG Support', 'Video Recording','Java']

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    #Features having very low variance 
    columns_to_remove = ['Architecture','Audio Jack','GPS','Loudspeaker','Network','Network Support','VoLTE']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    # Multivalued:
    columns_to_remove = ['Architecture','Launch Date','Audio Jack','GPS','Loudspeaker','Network','Network Support','VoLTE', 'Custom UI']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    # Not much important
    columns_to_remove = ['Bluetooth', 'Settings','Wi-Fi','Wi-Fi Features']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]
    
    return data

# Removing features

In [4]:
train = data_clean(train)
test = data_clean(test)

removing all those data points in which more than 15 features are missing 

In [5]:
train = train[(train.isnull().sum(axis=1) <= 15)]

# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 341
Number of features in train: 47
Number of data points in test: 119
Number of features in test: 46


# Filling Missing values

In [6]:
def for_integer(test):
    try:
        test = test.strip()
        return int(test.split(' ')[0])
    except IOError:
           pass
    except ValueError:
        pass
    except:
        pass

def for_string(test):
    try:
        test = test.strip()
        return (test.split(' ')[0])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass

def for_float(test):
    try:
        test = test.strip()
        return float(test.split(' ')[0])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass
def find_freq(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[2][0] == '(':
            return float(test[2][1:])
        return float(test[2])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass

    
def for_Internal_Memory(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[1] == 'GB':
            return int(test[0])
        if test[1] == 'MB':
#             print("here")
            return (int(test[0]) * 0.001)
    except IOError:
           pass
    except ValueError:
        pass
    except:
        pass
    
def find_freq(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[2][0] == '(':
            return float(test[2][1:])
        return float(test[2])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass


def data_clean_2(x):
    data = x.copy()
    
    data['Capacity'] = data['Capacity'].apply(for_integer)

    data['Height'] = data['Height'].apply(for_float)
    data['Height'] = data['Height'].fillna(data['Height'].mean())

    data['Internal Memory'] = data['Internal Memory'].apply(for_Internal_Memory)

    data['Pixel Density'] = data['Pixel Density'].apply(for_integer)

    data['Internal Memory'] = data['Internal Memory'].fillna(data['Internal Memory'].median())
    data['Internal Memory'] = data['Internal Memory'].astype(int)

    data['RAM'] = data['RAM'].apply(for_integer)
    data['RAM'] = data['RAM'].fillna(data['RAM'].median())
    data['RAM'] = data['RAM'].astype(int)

    data['Resolution'] = data['Resolution'].apply(for_integer)
    data['Resolution'] = data['Resolution'].fillna(data['Resolution'].median())
    data['Resolution'] = data['Resolution'].astype(int)

    data['Screen Size'] = data['Screen Size'].apply(for_float)

    data['Thickness'] = data['Thickness'].apply(for_float)
    data['Thickness'] = data['Thickness'].fillna(data['Thickness'].mean())
    data['Thickness'] = data['Thickness'].round(2)

    data['Type'] = data['Type'].fillna('Li-Polymer')

    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].apply(for_float)
    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].fillna(data['Screen to Body Ratio (calculated)'].mean())
    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].round(2)

    data['Width'] = data['Width'].apply(for_float)
    data['Width'] = data['Width'].fillna(data['Width'].mean())
    data['Width'] = data['Width'].round(2)

    data['Flash'][data['Flash'].isna() == True] = "Other"

    data['User Replaceable'][data['User Replaceable'].isna() == True] = "Other"

    data['Num_cores'] = data['Processor'].apply(for_string)
    data['Num_cores'][data['Num_cores'].isna() == True] = "Other"


    data['Processor_frequency'] = data['Processor'].apply(find_freq)
    #because there is one entry with 208MHz values, to convert it to GHz
    data['Processor_frequency'][data['Processor_frequency'] > 200] = 0.208
    data['Processor_frequency'] = data['Processor_frequency'].fillna(data['Processor_frequency'].mean())
    data['Processor_frequency'] = data['Processor_frequency'].round(2)

    data['Camera Features'][data['Camera Features'].isna() == True] = "Other"

    #simplifyig Operating System to os_name for simplicity
    data['os_name'] = data['Operating System'].apply(for_string)
    data['os_name'][data['os_name'].isna() == True] = "Other"

    data['Sim1'] = data['SIM 1'].apply(for_string)

    data['SIM Size'][data['SIM Size'].isna() == True] = "Other"

    data['Image Resolution'][data['Image Resolution'].isna() == True] = "Other"

    data['Fingerprint Sensor'][data['Fingerprint Sensor'].isna() == True] = "Other"

    data['Expandable Memory'][data['Expandable Memory'].isna() == True] = "No"

    data['Weight'] = data['Weight'].apply(for_integer)
    data['Weight'] = data['Weight'].fillna(data['Weight'].mean())
    data['Weight'] = data['Weight'].astype(int)

    data['SIM 2'] = data['SIM 2'].apply(for_string)
    data['SIM 2'][data['SIM 2'].isna() == True] = "Other"
    
    return data

In [7]:
train = data_clean_2(train)
test = data_clean_2(test)

# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 341
Number of features in train: 51
Number of data points in test: 119
Number of features in test: 50


Not very important feature

In [8]:
def data_clean_3(x):
    
    data = x.copy()

    columns_to_remove = ['User Available Storage','SIM Size','Chipset','Processor','Autofocus','Aspect Ratio','Touch Screen',
                        'Bezel-less display','Operating System','SIM 1','USB Connectivity','Other Sensors','Graphics','FM Radio',
                        'NFC','Shooting Modes','Browser','Display Colour' ]

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]


    columns_to_remove = [ 'Screen Resolution','User Replaceable','Camera Features',
                        'Thickness', 'Display Type']

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]


    columns_to_remove = ['Fingerprint Sensor', 'Flash', 'Rating Count', 'Review Count','Image Resolution','Type','Expandable Memory',\
                        'Colours','Width','Model']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    return data

train = data_clean_3(train)
test = data_clean_3(test)

# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

# one hot encoding

train_ids = train['PhoneId']
test_ids = test['PhoneId']

cols = list(test.columns)
cols.remove('PhoneId')
cols.insert(0, 'PhoneId')

combined = pd.concat([train.drop('Rating', axis=1)[cols], test[cols]])
print(combined.shape)
print(combined.columns)

combined = pd.get_dummies(combined)
print(combined.shape)
print(combined.columns)

train_new = combined[combined['PhoneId'].isin(train_ids)]
test_new = combined[combined['PhoneId'].isin(test_ids)]

train_new = train_new.merge(train[['PhoneId', 'Rating']], on='PhoneId')

# check the number of features and data points in train
print("Number of data points in train: %d" % train_new.shape[0])
print("Number of features in train: %d" % train_new.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test_new.shape[0])
print("Number of features in test: %d" % test_new.shape[1])

Number of data points in train: 341
Number of features in train: 18
Number of data points in test: 119
Number of features in test: 17
(460, 17)
Index(['PhoneId', 'Brand', 'Capacity', 'Screen to Body Ratio (calculated)',
       'Num_cores', 'Screen Size', 'os_name', 'Pixel Density', 'SIM 2',
       'Height', 'Sim1', 'Processor_frequency', 'Weight', 'Internal Memory',
       'Resolution', 'RAM', 'SIM Slot(s)'],
      dtype='object')
(460, 87)
Index(['PhoneId', 'Capacity', 'Screen to Body Ratio (calculated)',
       'Screen Size', 'Pixel Density', 'Height', 'Processor_frequency',
       'Weight', 'Internal Memory', 'Resolution', 'RAM', 'Brand_10.or',
       'Brand_Apple', 'Brand_Asus', 'Brand_Billion', 'Brand_Blackberry',
       'Brand_Comio', 'Brand_Coolpad', 'Brand_Do', 'Brand_Gionee',
       'Brand_Google', 'Brand_HTC', 'Brand_Honor', 'Brand_Huawei',
       'Brand_InFocus', 'Brand_Infinix', 'Brand_Intex', 'Brand_Itel',
       'Brand_Jivi', 'Brand_Karbonn', 'Brand_LG', 'Brand_Lava', 'Br

# Solution

In [9]:
# Last column contains the ratings i.e. the label
print(train_new.shape, test_new.shape)

(341, 88) (119, 87)


,PhoneId,Capacity,Screen to Body Ratio (calculated),Screen Size,Pixel Density,Height,Processor_frequency,Weight,Internal Memory,Resolution,...,SIM 2_4G,SIM 2_Other,Sim1_2G,Sim1_3G,Sim1_4G,"SIM Slot(s)_Dual SIM, GSM+CDMA","SIM Slot(s)_Dual SIM, GSM+GSM","SIM Slot(s)_Dual SIM, GSM+GSM, Dual VoLTE","SIM Slot(s)_Single SIM, GSM",Rating
0,0,4000,80.68,6.26,403,157.900000,1.80,182,64,20,...,1,0,0,0,1,0,0,1,0,1
1,1,4230,80.85,6.20,271,156.200000,1.80,168,32,8,...,1,0,0,0,1,0,0,1,0,1
2,2,3500,83.68,6.30,409,157.000000,2.10,168,32,25,...,1,0,0,0,1,0,0,1,0,1
3,4,3300,74.78,6.00,411,159.800000,2.20,169,64,24,...,1,0,0,0,1,0,1,0,0,1
4,5,3750,84.23,6.50,396,160.400000,2.20,175,64,16,...,1,0,0,0,1,0,0,1,0,1
5,7,3400,84.98,6.41,402,157.900000,2.20,156,64,25,...,1,0,0,0,1,0,0,1,0,1
6,8,3000,72.68,5.45,295,147.500000,2.00,145,16,5,...,1,0,0,0,1,0,0,1,0,1
7,9,4000,79.24,5.84,432,149.300000,2.00,178,32,5,...,1,0,0,0,1,0,1,0,0,1
8,10,4230,80.85,6.20,271,156.200000,1.80,168,16,8,...,1,0,0,0,1,0,1,0,0,1
9,12,4000,82.20,6.18,403,155.500000,2.80,182,64,20,...,1,0,0,0,1,0,0,1,0,1


In [10]:
# Labels have not been binarized
train_new['Rating'][25:40]
train_new.drop('Weight', axis = 1)

25    4.3
26    4.3
27    4.6
28    4.4
29    4.5
30    3.9
31    4.2
32    4.3
33    4.2
34    4.3
35    4.3
36    4.2
37    4.4
38    4.1
39    3.8
Name: Rating, dtype: float64

### Binarize the target labels

In [11]:
train_new['Rating'] = train_new['Rating'].apply(lambda x: 1 if x>=4 else 0)

In [12]:
train_new['Rating'][25:40]

25    1
26    1
27    1
28    1
29    1
30    0
31    1
32    1
33    1
34    1
35    1
36    1
37    1
38    1
39    0
Name: Rating, dtype: int64

### Split train_new into X and y.

In [13]:
y = train_new['Rating']
X = train_new.drop('Rating', axis = 1)  # Not inplace, train_new is not affected

In [14]:
print(X.shape, y.shape)

(341, 87) (341,)


In [15]:
# Verifying that y contains the labels. Also data imbalance is observed hence stratified split is necessary in next step
y.value_counts()

1    238
0    103
Name: Rating, dtype: int64

### Split the training data into training and validation set

In [16]:
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state = 42, stratify = y, test_size = 0.1)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)
print(y_train.mean(), y_val.mean())

(306, 87) (35, 87) (306,) (35,)
0.6993464052287581 0.6857142857142857


### Standardizing the data

Standardization should not be done for one hot encoded columns

In [17]:
# Find all columns which are one-hot encoded
not_onehot = list()
onehot = list()
for i in X_train.columns:
    if X_train[i].dtype != 'uint8':
        not_onehot.append(i)
    else:
        onehot.append(i)

Standardizing by **median** instead of mean, makes it **robust to outliers**.

In [18]:
X_train[not_onehot]=(X_train[not_onehot]-X_train[not_onehot].median())/X_train[not_onehot].std()
X_val[not_onehot]=(X_val[not_onehot]-X_val[not_onehot].median())/X_val[not_onehot].std()

In [19]:
X_train.head(10)

,PhoneId,Capacity,Screen to Body Ratio (calculated),Screen Size,Pixel Density,Height,Processor_frequency,Weight,Internal Memory,Resolution,...,SIM 2_3G,SIM 2_4G,SIM 2_Other,Sim1_2G,Sim1_3G,Sim1_4G,"SIM Slot(s)_Dual SIM, GSM+CDMA","SIM Slot(s)_Dual SIM, GSM+GSM","SIM Slot(s)_Dual SIM, GSM+GSM, Dual VoLTE","SIM Slot(s)_Single SIM, GSM"
39,-1.275027,0.990322,-0.161122,0.000000,0.849371,0.331501,1.119537,0.835420,0.671006,0.845493,...,0,1,0,0,0,1,0,1,0,0
41,-1.260244,0.463555,1.194382,0.921066,2.321615,0.716471,2.015167,1.231146,0.671006,0.000000,...,0,1,0,0,0,1,0,0,1,0
26,-1.408074,0.779616,0.869019,1.052647,0.747447,1.186989,0.895630,0.967329,2.013018,2.705578,...,0,1,0,0,0,1,0,1,0,0
254,0.905454,0.990322,0.281704,0.263162,-0.498298,0.320808,-0.895630,0.615573,0.000000,0.000000,...,0,1,0,0,0,1,0,1,0,0
295,1.341550,-0.063212,-0.316007,0.000000,-0.668172,0.106936,-0.447815,0.395725,0.000000,-0.507296,...,1,0,0,0,0,1,0,1,0,0
117,-0.528490,0.147495,-0.034303,0.000000,0.849371,0.203178,-0.671722,-0.351756,0.671006,1.352789,...,0,1,0,0,0,1,0,1,0,0
1,-1.681558,1.232635,0.867980,0.921066,-0.622872,0.513292,0.000000,0.307786,0.000000,0.000000,...,0,1,0,0,0,1,0,0,1,0
135,-0.328920,0.990322,0.293138,0.657904,0.464323,1.229763,0.335861,1.187176,0.671006,1.352789,...,0,1,0,0,0,1,0,1,0,0
58,-1.127198,0.463555,1.304568,1.184228,0.872021,0.652309,0.447815,0.923359,2.013018,2.874676,...,0,1,0,0,0,1,0,0,1,0
122,-0.491532,-1.305329,-0.755714,-1.052647,0.000000,-1.390167,1.321054,-0.571603,0.671006,-0.169099,...,0,0,1,0,0,1,0,0,0,1


In [20]:
train_new.iloc[[39, 41, 26, 254, 295, 117, 1, 135, 58, 122], :]

,PhoneId,Capacity,Screen to Body Ratio (calculated),Screen Size,Pixel Density,Height,Processor_frequency,Weight,Internal Memory,Resolution,...,SIM 2_4G,SIM 2_Other,Sim1_2G,Sim1_3G,Sim1_4G,"SIM Slot(s)_Dual SIM, GSM+CDMA","SIM Slot(s)_Dual SIM, GSM+GSM","SIM Slot(s)_Dual SIM, GSM+GSM, Dual VoLTE","SIM Slot(s)_Single SIM, GSM",Rating
39,56,4000,70.95,5.5,401,154.5,2.30,180,64,13,...,1,0,0,0,1,0,1,0,0,0
41,58,3500,83.99,6.2,531,158.1,2.70,189,64,8,...,1,0,0,0,1,0,0,1,0,1
26,38,3800,80.86,6.3,392,162.5,2.20,183,128,24,...,1,0,0,0,1,0,1,0,0,1
254,351,4000,75.21,5.7,282,154.4,1.40,175,32,8,...,1,0,0,0,1,0,1,0,0,1
295,410,3000,69.46,5.5,267,152.4,1.60,170,32,5,...,0,0,0,0,1,0,1,0,0,0
117,157,3200,72.17,5.5,401,153.3,1.50,153,64,16,...,1,0,0,0,1,0,1,0,0,1
1,1,4230,80.85,6.2,271,156.2,1.80,168,32,8,...,1,0,0,0,1,0,0,1,0,1
135,184,4000,75.32,6.0,367,162.9,1.95,188,64,16,...,1,0,0,0,1,0,1,0,0,1
58,76,3500,85.05,6.4,403,157.5,2.00,182,128,25,...,1,0,0,0,1,0,0,1,0,1
122,162,1821,65.23,4.7,326,138.4,2.39,148,64,7,...,0,1,0,0,1,0,0,0,1,1


## Remove unneccessary one-hot columns

Many **one-hot** encoded columns contain **very few positive cases** i.e. value is 1. 

*Hypothesis:* these features will not help the mp neuron model to generalize. As the drawback in mp neuron model is that **all columns/features are given equal weight**.

In [21]:
for col in onehot[:10]:
    temp = X[col].value_counts().values
    try:
        print('0:', temp[0], '; 1:', temp[1])
    except:
        print('0:', temp[0], '; 1: 0')

0: 339 ; 1: 2
0: 322 ; 1: 19
0: 333 ; 1: 8
0: 341 ; 1: 0
0: 336 ; 1: 5
0: 340 ; 1: 1
0: 339 ; 1: 2
0: 341 ; 1: 0
0: 331 ; 1: 10
0: 337 ; 1: 4


In [22]:
PERCENTAGE = 0.000

print('Original shape:', X_train.shape, X_val.shape)

one_hot_drop = list()
for i in X_train.columns:
    if X_train[i].dtype == 'uint8':
        if X_train[i].shape[0]*PERCENTAGE > X_train[i].sum(axis=0):
            one_hot_drop.append(i)

X_train.drop(one_hot_drop, axis = 1, inplace=True)
X_val.drop(one_hot_drop, axis = 1, inplace=True)

# Removing PhoneId as i has no predictive power
X_train.drop('PhoneId', axis = 1, inplace=True)
X_val.drop('PhoneId', axis = 1, inplace=True)

print(f'After removing {PERCENTAGE*100}% columns:', X_train.shape, X_val.shape)

Original shape: (306, 87) (35, 87)
After removing 0.0% columns: (306, 86) (35, 86)


## Removing uncorrelated columns

In [23]:
absCorrWithTar = list()
target = y_train
for col in X_train.columns:
    feature  = X_train[col]
    absCorrWithTar.append((col, abs(target.corr(feature))))

absCorrWithTar.sort(key = lambda x: x[1])

NUM_COLUMNS_DROP = 15
remove_cols = [i[0] for i in absCorrWithTar[:NUM_COLUMNS_DROP]]
print('Removed columns:\n', remove_cols)

print(X_train.shape, X_val.shape)

columns_to_retain = list(set(X_train.columns)-set(remove_cols))


X_train = X_train[columns_to_retain]
X_val = X_val[columns_to_retain]

print(X_train.shape, X_val.shape)

Removed columns:
 ['os_name_Other', 'Brand_Itel', 'Brand_Moto', 'Brand_Lava', 'Brand_Sony', 'Brand_Infinix', 'Brand_Ulefone', 'Brand_LeEco', 'Brand_Motorola', 'Num_cores_312', 'SIM Slot(s)_Dual SIM, GSM+CDMA', 'os_name_Nokia', 'Brand_Comio', 'Num_cores_Tru-Octa', 'Brand_Gionee']
(306, 86) (35, 86)
(306, 71) (35, 71)


In [24]:
print(absCorrWithTar[:20])

[('os_name_Other', 0.004984993126236153), ('Brand_Itel', 0.01271327443565058), ('Brand_Moto', 0.026974855199058345), ('Brand_Lava', 0.02792380121235746), ('Brand_Sony', 0.02792380121235759), ('Brand_Infinix', 0.03526202432508638), ('Brand_Ulefone', 0.03754370314827091), ('Brand_LeEco', 0.037543703148270914), ('Brand_Motorola', 0.037543703148270914), ('Num_cores_312', 0.03754370314827093), ('SIM Slot(s)_Dual SIM, GSM+CDMA', 0.03754370314827095), ('os_name_Nokia', 0.037543703148270956), ('Brand_Comio', 0.03754370314827096), ('Num_cores_Tru-Octa', 0.03754370314827096), ('Brand_Gionee', 0.039822353911713515), ('Num_cores_Dual', 0.03982235391171358), ('Brand_Oppo', 0.04132359097499468), ('Brand_Asus', 0.041323590974994724), ('Num_cores_Other', 0.04132359097499473), ('Brand_Nokia', 0.047535364006418425)]


## Perceptron Class

In [25]:
class Perceptron:
  
    def __init__ (self):
        self.w = None
        self.b = None

    def model(self, x):
        return 1 if (np.dot(self.w, x) >= self.b) else 0

    def predict(self, X):
        Y = []
        for x in X:
            result = self.model(x)
            Y.append(result)
        return np.array(Y)

    def fit(self, X, Y, epochs = 1, lr = 1):

        self.b = 0
        self.w = np.ones(X.shape[1])
        accuracy = {}
        max_accuracy = 0

        for i in range(epochs):
            for x, y in zip(X, Y):
                y_pred = self.model(x)
                if y == 1 and y_pred == 0:
                    self.w = self.w + lr * x
                    self.b = self.b - lr * 1
                elif y == 0 and y_pred == 1:
                    self.w = self.w - lr * x
                    self.b = self.b + lr * 1 

            accuracy[i] = accuracy_score(self.predict(X), Y)
            if (accuracy[i] > max_accuracy):
                max_accuracy = accuracy[i]
                chkptw = self.w
                chkptb = self.b

        self.w = chkptw
        self.b = chkptb

        print('Max training accuracy:',max_accuracy)
        
        return max_accuracy, (chkptw, chkptb)

## Grid Search on LR and Epochs

In [26]:
val_accs = list()

epochs = [5000, 10000]
LR = [0.001, 0.01, 0.1]

for i, j in list(itertools.product(epochs, LR)):
    perceptron = Perceptron()
    print('epochs:', i, ' ,LR:', j)
    
    train_score, wt_matrix = perceptron.fit(X_train.values, y_train.values, i, j)
    
    Y_pred_test = perceptron.predict(X_val.values)
    
    val_score = accuracy_score(Y_pred_test, y_val.values)
    print('Max Validation accuracy:', val_score)
    print('-'*50)
    val_accs.append((train_score, val_score, i, j, wt_matrix))
    
max_val = 0
Final_config = None
for i in val_accs:
    if i[1]>max_val:
        Final_config = i
        max_val = i[1]

print(Final_config[:-1])
    

epochs: 5000  ,LR: 0.001
Max training accuracy: 0.8594771241830066
Max Validation accuracy: 0.42857142857142855
--------------------------------------------------
epochs: 5000  ,LR: 0.01
Max training accuracy: 0.8627450980392157
Max Validation accuracy: 0.7428571428571429
--------------------------------------------------
epochs: 5000  ,LR: 0.1
Max training accuracy: 0.8627450980392157
Max Validation accuracy: 0.6285714285714286
--------------------------------------------------
epochs: 10000  ,LR: 0.001
Max training accuracy: 0.869281045751634
Max Validation accuracy: 0.42857142857142855
--------------------------------------------------
epochs: 10000  ,LR: 0.01
Max training accuracy: 0.8660130718954249
Max Validation accuracy: 0.6571428571428571
--------------------------------------------------
epochs: 10000  ,LR: 0.1
Max training accuracy: 0.8627450980392157
Max Validation accuracy: 0.6285714285714286
--------------------------------------------------
(0.8627450980392157, 0.7428571

## Final model

Using the best LR and epoch combination from the grid search, we train the final model.

In [27]:
perceptron = Perceptron()

train_score, wt_matrix = perceptron.fit(X_train.values, y_train.values, 5000, 0.01)

Y_pred_test = perceptron.predict(X_val.values) 
val_score = accuracy_score(Y_pred_test, y_val.values)

print('Max Validation accuracy:', val_score)

Max training accuracy: 0.8627450980392157
Max Validation accuracy: 0.7428571428571429


## Submission of csv file

Same transformation have to be performed on test set as the ones performed on training set. This is the **preprocessing step** before using our model for prediction.

In [28]:
test_new.head()

,PhoneId,Capacity,Screen to Body Ratio (calculated),Screen Size,Pixel Density,Height,Processor_frequency,Weight,Internal Memory,Resolution,...,SIM 2_3G,SIM 2_4G,SIM 2_Other,Sim1_2G,Sim1_3G,Sim1_4G,"SIM Slot(s)_Dual SIM, GSM+CDMA","SIM Slot(s)_Dual SIM, GSM+GSM","SIM Slot(s)_Dual SIM, GSM+GSM, Dual VoLTE","SIM Slot(s)_Single SIM, GSM"
0,3,4230,80.85,6.20,271,156.2,1.8,168,16,5,...,0,1,0,0,0,1,0,0,1,0
1,11,5000,81.60,6.20,402,156.0,1.8,205,64,12,...,0,1,0,0,0,1,0,0,1,0
2,13,3500,83.84,6.30,409,156.7,2.0,169,64,25,...,0,1,0,0,0,1,0,0,1,0
3,16,3500,83.84,6.30,409,156.7,2.0,169,64,16,...,0,1,0,0,0,1,0,0,1,0
4,19,4000,77.43,5.99,403,158.6,1.8,181,64,20,...,0,1,0,0,0,1,0,0,1,0


In [29]:
## Submission_dropcol0.005_median    --> LB 0.88571
## drop columns percentage = 0.005, dropped phoneid, epochs = 10K , LR = 0.01, median normalization

# Drop PhoneId in test set as well as the insignificant one hot encoded columns. Percentage to drop must be same.
temp = test_new['PhoneId']
test_new[not_onehot]=(test_new[not_onehot]-test_new[not_onehot].median())/test_new[not_onehot].std()
test_new.drop(one_hot_drop, axis = 1, inplace=True)
test_new.drop('PhoneId', axis = 1, inplace=True)

In [30]:
# Predict using model
test_new_preds = perceptron.predict(test_new.values)

# Create dataframe
submission = pd.DataFrame({'PhoneId':temp, 'Class':test_new_preds})
submission = submission[['PhoneId', 'Class']]
submission.head()

# Write to csv file
submission.to_csv("submission.csv", index=False)

ValueError: shapes (71,) and (86,) not aligned: 71 (dim 0) != 86 (dim 0)